## Weaviate 1.32 enablement session

### Agenda

- Collection aliases
- Replica movement
- Rotational quantization
- Python: Vectorizer DX refactor
- HNSW connection compression
- RBAC updates
- New models

### Prep

In [1]:
import weaviate
import os

client = weaviate.connect_to_local(
    headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"),
    }
)

client.is_ready()

True

In [2]:
client.collections.delete_all()

### Collection aliases

#### How things work now

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

client.collections.create(
    "MoviesConfigA",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(name="default"),
    ],
    generative_config=Configure.Generative.cohere()
)

In [ ]:
objects = [
    {
        "title": "Tropic Thunder",
        "body": "When their frustrated director (Steve Coogan) drops them in the middle of a jungle and dies in an accident, they are forced to rely on their acting skills to survive the real action and danger."
    },
    {
        "title": "Dodgeball",
        "body": "A group of unlikely misfits enter a Las Vegas dodgeball tournament, needing the prize money to save their cherished gym from being taken over by a corporate health fitness chain."
    },
    {
        "title": "Zoolander",
        "body": "Derek Zoolander (Stiller) is tricked by fashion mogul Jacobim Mugatu (Will Ferrell) into assassinating the Prime Minister of Malaysia, whose progressive laws on the fashion industry would harm his businesses.",
    },
    {
        "title": "Blades of Glory",
        "body": "A mismatched pair of banned figure skaters become teammates upon discovering a loophole that will allow them to compete in the sport again."
    }
]

In [ ]:
movies_direct = client.collections.get("MoviesConfigA")

movies_direct.data.insert_many(objects=objects)

BatchObjectReturn(_all_responses=[UUID('5688d747-966b-4730-ab91-489a1df4e0f1'), UUID('29e8937c-bf69-401e-89d5-e87709629a09'), UUID('cc062d0d-cfbd-4bb2-a142-3be047ee45fa'), UUID('c2a6388d-3730-441d-b318-47fe7037dcd1')], elapsed_seconds=0.30062079429626465, errors={}, uuids={0: UUID('5688d747-966b-4730-ab91-489a1df4e0f1'), 1: UUID('29e8937c-bf69-401e-89d5-e87709629a09'), 2: UUID('cc062d0d-cfbd-4bb2-a142-3be047ee45fa'), 3: UUID('c2a6388d-3730-441d-b318-47fe7037dcd1')}, has_errors=False)

![img/1_32_aliases/1_collection_before.png](img/1_32_aliases/1_collection_before.png)

In [ ]:
r = movies_direct.query.hybrid(
    query="sports comedy",
    limit=2
)

for o in r.objects:
    print(o.properties["title"])

Dodgeball
Blades of Glory


#### Introducing: Aliases

In [ ]:
client.alias.create(alias_name="Movies", target_collection="MoviesConfigA")

In [ ]:
movies = client.collections.get("Movies")  # <-- Treat the alias as a collection

r = movies.query.hybrid(
    query="sports comedy",
    limit=2
)

for o in r.objects:
    print(o.properties["title"])

Dodgeball
Blades of Glory


![img/1_32_aliases/2_collection_with_alias.png](img/1_32_aliases/2_collection_with_alias.png)

#### Why aliases, though?

What if your needs change?

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

client.collections.delete("MoviesConfigB")

client.collections.create(
    "MoviesConfigB",
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
        Property(name="rating", data_type=DataType.NUMBER),
        Property(name="release_year", data_type=DataType.INT),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            name="default",
            vector_index_config=Configure.VectorIndex.hnsw(
                quantizer=Configure.VectorIndex.Quantizer.sq()
            ),
        ),
    ],
    generative_config=Configure.Generative.anthropic()
)

In [ ]:
objects = [
    {
        "title": "Tropic Thunder",
        "body": "When their frustrated director (Steve Coogan) drops them in the middle of a jungle and dies in an accident, they are forced to rely on their acting skills to survive the real action and danger.",
        "rating": 9.0,
        "release_year": 2008
    },
    {
        "title": "Dodgeball",
        "body": "A group of unlikely misfits enter a Las Vegas dodgeball tournament, needing the prize money to save their cherished gym from being taken over by a corporate health fitness chain.",
        "rating": 8.5,
        "release_year": 2004
    },
    {
        "title": "Zoolander",
        "body": "Derek Zoolander (Stiller) is tricked by fashion mogul Jacobim Mugatu (Will Ferrell) into assassinating the Prime Minister of Malaysia, whose progressive laws on the fashion industry would harm his businesses.",
        "rating": 7.5,
        "release_year": 2001
    },
    {
        "title": "Blades of Glory",
        "body": "A mismatched pair of banned figure skaters become teammates upon discovering a loophole that will allow them to compete in the sport again.",
        "rating": 8.0,
        "release_year": 2007
    }
]

In [ ]:
movies = client.collections.get("MoviesConfigB")

movies.data.insert_many(objects=objects)

BatchObjectReturn(_all_responses=[UUID('9c1d5a41-3948-40ac-8324-579fc843ccb5'), UUID('05e6bcde-63a9-485e-bf36-743cb62022cf'), UUID('b6629976-5589-4550-8df5-be6104259c51'), UUID('c9823405-9d3f-427e-820f-10a29dc53a3b')], elapsed_seconds=0.1859748363494873, errors={}, uuids={0: UUID('9c1d5a41-3948-40ac-8324-579fc843ccb5'), 1: UUID('05e6bcde-63a9-485e-bf36-743cb62022cf'), 2: UUID('b6629976-5589-4550-8df5-be6104259c51'), 3: UUID('c9823405-9d3f-427e-820f-10a29dc53a3b')}, has_errors=False)

![img/1_32_aliases/3_collection_to_another.png](img/1_32_aliases/3_collection_to_another.png)

Now, with aliases... this becomes super easy!

In [ ]:
client.alias.update(alias_name="Movies", new_target_collection="MoviesConfigB")

True

In [ ]:
movies = client.collections.get("Movies")

r = movies.query.hybrid(
    query="sports comedy",
    limit=2
)

for o in r.objects:
    print(o.properties["title"])

Dodgeball
Blades of Glory


In [ ]:
from weaviate.classes.query import Filter

r = movies.query.hybrid(
    query="sports comedy",
    limit=2,
    filters=Filter.by_property("release_year").greater_than(2006)
)

for o in r.objects:
    print(o.properties["title"])

Blades of Glory
Tropic Thunder


![img/1_32_aliases/4_collection_alias_swap.png](img/1_32_aliases/4_collection_alias_swap.png)

In [ ]:
client.alias.list_all()

<style>
.admonition {
    padding: 10px;
    margin: 10px; 
    border-left: 4px solid;
    border-radius: 4px;
}
.note { border-color: #007acc; background-color: #f0f8ff; }
.warning { border-color: #ff6b6b; background-color: #fff5f5; }
</style>

<div class="admonition note">
<strong>Note:</strong> Collection aliases are in technical preview in Weaviate 1.32
</div>

### Shard replica movement

#### What are shards?

![img/1_32_shard_movement/1_what_are_shards.png](img/1_32_shard_movement/1_what_are_shards.png)

#### Shards in a multi-node cluster

- S1, S2, etc -> shard replicas

![img/1_32_shard_movement/2_shards_before.png](img/1_32_shard_movement/2_shards_before.png)

#### Shard movement in a multi-node cluster

1. If some nodes are more loaded than others 
- e.g. due to some nodes having more data, seeing more queries, etc.

![img/1_32_shard_movement/3_shard_replica_movement.png](img/1_32_shard_movement/3_shard_replica_movement.png)

Move shards between nodes to balance the load

2. If cluster is reaching its capacity
- Too much data / too many queries, etc. overall

![img/1_32_shard_movement/4_shard_replica_movement_to_new_node.png](img/1_32_shard_movement/4_shard_replica_movement_to_new_node.png)

#### Replica movement syntax

In [ ]:
collection_name = "CollectionWithReplicas"

collection_sharding_state = client.cluster.query_sharding_state(
    collection=collection_name
)
if collection_sharding_state and collection_sharding_state.shards:
    print(f"Shards in '{collection_name}': {[s.name for s in collection_sharding_state.shards]}")

In [ ]:
# Get 'shard_name' above ⬆️

shard_name = "tF9DtxC59ykC"
specific_shard_state = client.cluster.query_sharding_state(collection=collection_name, shard=shard_name)

if specific_shard_state and specific_shard_state.shards:
    print(f"Nodes for shard '{shard_name}': {specific_shard_state.shards[0].replicas}")

Initiate shard movement

In [ ]:
from weaviate.cluster.models import TransferType

operation_id = client.cluster.replicate(
    collection=collection_name,
    shard=shard_name,
    source_node=source_node_name,
    target_node=target_node_name,
    transfer_type=TransferType.COPY,  # or TransferType.MOVE
)
print(f"Replication initiated, ID: {operation_id}")

Check operation status

In [ ]:
op_status = client.cluster.replications.get(
    uuid=operation_id,
    # include_history=True
)
print(f"Status for {operation_id}: {op_status.status.state}")

Operations can be:
- Cancelled
- Deleted (cancel, then delete)

Can also list all operations.

### Rotational quantization

New quantization method for vectors. RQ:

- Does not require training
- Increases query speed
- Has negligible impact on search quality

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

collection_name = "Movies"

client.collections.delete(collection_name)

client.collections.create(
    collection_name,
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
    ],
    vectorizer_config=[
        Configure.NamedVectors.text2vec_cohere(
            vector_index_config=Configure.VectorIndex.hnsw(
                # ===== Enable RQ like this =====
                quantizer=Configure.VectorIndex.Quantizer.rq()
                # ===== 🎉 That's it! =====
            )
        )
    ]
)

![img/1_32_rq/rq-test-results.png](img/1_32_rq/rq-test-results.png)

<style>
.admonition {
    padding: 10px;
    margin: 10px; 
    border-left: 4px solid;
    border-radius: 4px;
}
.note { border-color: #007acc; background-color: #f0f8ff; }
.warning { border-color: #ff6b6b; background-color: #fff5f5; }
</style>

<div class="admonition note">
<strong>Note:</strong> Rotational quantization is in technical preview in Weaviate 1.32
</div>

Coming soon (1.33?) - further quantization improvements

- Lower bit precision
- Nudges to make it easier to use quantization

### HNSW connection compression

![img/1_32_hnsw/hnsw-overall.png](img/1_32_hnsw/hnsw-overall.png)

#### Connections

![img/1_32_hnsw/hnsw-params.png](img/1_32_hnsw/hnsw-params.png)

Lots of connections between nodes in HNSW
- Each node (vector) will typically have ~32/64 connections
- At millions of vectors, this can be a lot of memory
    - Especially for multi-vector embeddings

**Connection compression reduces the memory footprint of these connections**

#### User interface for compressed connections

Nothing! 😜

Just works out of the box, from 1.32 onwards.

### Vectorizer refactor

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

collection_name = "Movies"

client.collections.delete(collection_name)

client.collections.create(
    collection_name,
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
    ],
    vector_config=Configure.Vectors.text2vec_cohere()
)

In [ ]:
from weaviate.classes.config import Configure, Property, DataType

collection_name = "Movies"

client.collections.delete(collection_name)

client.collections.create(
    collection_name,
    properties=[
        Property(name="title", data_type=DataType.TEXT),
        Property(name="body", data_type=DataType.TEXT),
    ],
    vector_config=[
        Configure.Vectors.text2vec_cohere(
            name="default"
        ),
        Configure.Vectors.text2vec_cohere(
            name="title",
            source_properties=["title"],
        ),
    ]
)

In [13]:
objects = [
    {
        "title": "Police Academy",
        "body": "A group of misfits enter a police academy and become friends while training"
    },
    {
        "title": "Police Academy 2",
        "body": "The misfits from the first movie are now police officers and must stop a gang of criminals"
    },
    {
        "title": "Police Academy 3",
        "body": "The misfits are now instructors at the police academy and must train a new group of recruits"
    }
]

In [14]:
movies = client.collections.get(collection_name)

movies.data.insert_many(objects=objects)

BatchObjectReturn(_all_responses=[UUID('9b30cfbb-2798-4c79-bb7f-828b92e9b5c2'), UUID('c15d4c6d-9f85-454b-9bd7-9d6bf0dce2bf'), UUID('f2d74b09-70c2-4fab-b032-8ca5fb0318b1')], elapsed_seconds=0.30727601051330566, errors={}, uuids={0: UUID('9b30cfbb-2798-4c79-bb7f-828b92e9b5c2'), 1: UUID('c15d4c6d-9f85-454b-9bd7-9d6bf0dce2bf'), 2: UUID('f2d74b09-70c2-4fab-b032-8ca5fb0318b1')}, has_errors=False)

In [15]:
response = movies.query.fetch_objects(
    limit=3,
    include_vector=True
)

for o in response.objects:
    print(f"\nObject ID: {o.uuid}")
    for k, v in o.vector.items():
        print(f"Vector {k}: {v[:3]}...")


Object ID: 9b30cfbb-2798-4c79-bb7f-828b92e9b5c2
Vector default: [0.01995849609375, 0.037750244140625, 0.01806640625]...

Object ID: c15d4c6d-9f85-454b-9bd7-9d6bf0dce2bf
Vector default: [0.01316070556640625, 0.04937744140625, -0.030609130859375]...

Object ID: f2d74b09-70c2-4fab-b032-8ca5fb0318b1
Vector default: [0.021026611328125, 0.02020263671875, -0.0177154541015625]...


### Others

#### RBAC

- Option to include RBAC roles & users to Weaviate backups

#### New model integrations

Support added for:

- Gemini embedding models ([Google docs](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/text-embeddings-api))
  - `gemini-embedding-001`
  - `text-embedding-005`
  - `text-multilingual-embedding-002`

- Transformers models
  - `intfloat/multilingual-e5-large`
  - `Qwen/Qwen3-Embedding-0.6B` 
  - `Qwen/Qwen3-Embedding-4B` 